___


# <font color= #8FC3FA> **NYC Taxi Predictions 2025 - Model Experiments 2** </font>
#### <font color= #2E9AFE> `Data Science Project - Homework 5`</font>
- <Strong> Viviana Toledo </Strong>
- <Strong> Fecha: </Strong> 28/10/2025

___

In [20]:
# General Libraries
import pandas as pd
from datetime import datetime

# Databricks Env
from dotenv import load_dotenv
import pickle
import pathlib

# Feature Engineering
from sklearn.feature_extraction import DictVectorizer

# Optimization
import math
import optuna
from optuna.samplers import TPESampler

# Modeling
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# MLFlow
import mlflow
import mlflow.pyfunc
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow import MlflowClient

# Evaluation
from sklearn.metrics import root_mean_squared_error

# Autolog function
mlflow.sklearn.autolog()

In [21]:
load_dotenv(override=True)  # Carga las variables del archivo .env
EXPERIMENT_NAME = "/Users/viviana.toledo@iteso.mx/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

# <font color= #8FC3FA> **1. Data Loading** </font>

First of all, we'll start by loading the data:

In [29]:
def read_dataframe(path):
    df = pd.read_parquet(path)
    df["duration"] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
    return df

The data is stored in .parquet files. Our function defined above helps us handle these types of data.

In [30]:
df_train = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2025-02.parquet')

For modeling, we create a new variable composed of two variables in the dataset:

In [31]:
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

# <font color= #8FC3FA> **2. Feature Engineering** </font>

Afterwards, we will proceed to apply feature engineering, which includes dividing features by categorical and numerical types:

In [32]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [33]:
# Dictionaries for preprocessing
dv = DictVectorizer()

# Define categorical and numerical variables
categorical = ['PU_DO']
numerical = ['trip_distance']

# Fit DictVectorizer on training data
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Validation
X_val = preprocess(df_val, dv)

And our target variables:

In [34]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Upload the datasets to mlflow:

In [35]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

# <font color= #8FC3FA> **3. Modeling** </font>

## <font color= #8FC3FA> &ensp; • **Random Forest** </font>

We're going to perform a hyperparameter search for our Random Forest Regressor Model using Optuna. Firstly, we have to define the target function:

In [38]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna
#    - Recibe un `trial`, que se usa para proponer hiperparámetros.
#    - Entrena un modelo con esos hiperparámetros.
#    - Calcula la métrica de validación (RMSE) y la retorna (Optuna la minimizará).
#    - Abrimos un run anidado de MLflow para registrar cada trial.
# ------------------------------------------------------------

def objective(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 4, 100),
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "min_samples_split": trial.suggest_int("min_samples_split", 4, 100),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 4, 100),
        "min_weight_fraction_leaf": trial.suggest_float("min_weight_fraction_leaf", math.exp(-3), math.exp(-1), log=True),
        "max_features": trial.suggest_int("max_features", 4, 100),
        "ccp_alpha": trial.suggest_float("ccp_alpha",   math.exp(-3), math.exp(-3), log=True),
        "random_state": 42,                      
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "randomforest")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento con el conjunto de validación
        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)

        # Predicción y métrica en validación
        y_pred = rf.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val, y_pred)

        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.sklearn.log_model(
            sk_model = rf,
            name="model",
            input_example=X_val[:5],
            signature=signature,
        )

    # Optuna minimiza el valor retornado
    return rmse

Now, we can execute the search and log the models into MLFlow:

In [39]:
mlflow.sklearn.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="RandomForest Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=10)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "randomforest",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------

    # Select parameters
    rf = RandomForestRegressor(
        n_estimators=best_params["n_estimators"],
        max_depth=best_params["max_depth"],
        min_samples_split=best_params["min_samples_split"],
        min_samples_leaf=best_params["min_samples_leaf"],
        min_weight_fraction_leaf=best_params["min_weight_fraction_leaf"],
        max_features=best_params["max_features"],
        ccp_alpha=best_params["ccp_alpha"],
        random_state=42
    )
    # Fit the model
    rf.fit(X_train, y_train)

    # Evaluar y registrar la métrica final en validación
    y_pred = rf.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

    # Para que las longitudes coincidan, usa el mismo slice en y_pred
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.sklearn.log_model(
    sk_model=rf,                    # Trained RandomForestRegressor
    name="model",                   # Folder inside MLflow run to store the model
    input_example= input_example,   # First few rows of validation data
    signature=signature,            
)

[I 2025-10-23 21:39:04,957] A new study created in memory with name: no-name-42889371-c6b2-4cfa-809e-38de79171c79
2025/10/23 21:39:19 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/10/23 21:39:19 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run stylish-carp-81 at: https://dbc-ce601eea-1bde.cloud.databricks.com/ml/experiments/2243674360324792/runs/b8475a0ae92f4d5396ba3bc3d6eb35c0
🧪 View experiment at: https://dbc-ce601eea-1bde.cloud.databricks.com/ml/experiments/2243674360324792


[I 2025-10-23 21:39:22,342] Trial 0 finished with value: 9.106328108013187 and parameters: {'n_estimators': 40, 'max_depth': 96, 'min_samples_split': 75, 'min_samples_leaf': 62, 'min_weight_fraction_leaf': 0.06801937287807802, 'max_features': 19, 'ccp_alpha': 0.049787068367863944}. Best is trial 0 with value: 9.106328108013187.
2025/10/23 21:39:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/10/23 21:39:35 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


...

2025/10/23 21:41:42 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run able-quail-95 at: https://dbc-ce601eea-1bde.cloud.databricks.com/ml/experiments/2243674360324792/runs/1e0df3229eac4d789fb4b6eea9654c5d
🧪 View experiment at: https://dbc-ce601eea-1bde.cloud.databricks.com/ml/experiments/2243674360324792


[I 2025-10-23 21:41:44,574] Trial 9 finished with value: 9.106439957644191 and parameters: {'n_estimators': 61, 'max_depth': 93, 'min_samples_split': 12, 'min_samples_leaf': 23, 'min_weight_fraction_leaf': 0.05450049895708781, 'max_features': 35, 'ccp_alpha': 0.049787068367863944}. Best is trial 8 with value: 8.998080851599308.
2025/10/23 21:41:57 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\Vivienne\apps\data_science_project\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
2025/10/23 21:42:00 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run RandomForest Hyperparameter Optimization (Optuna) at: https://dbc-ce601eea-1bde.cloud.databricks.com/ml/experiments/2243674360324792/runs/260a6b2f1c1240e6a7c894f42ee00f07
🧪 View experiment at: https://dbc-ce601eea-1bde.cloud.databricks.com/ml/experiments/2243674360324792
